# Neural Network - Multi Layer

In [1]:
import numpy as np
import tensorflow as tf

------------------------------------------------------------
## 1. Data (mnist)


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("C:/Users/06411/tmp/data/", one_hot=True)

Extracting C:/Users/06411/tmp/data/train-images-idx3-ubyte.gz
Extracting C:/Users/06411/tmp/data/train-labels-idx1-ubyte.gz
Extracting C:/Users/06411/tmp/data/t10k-images-idx3-ubyte.gz
Extracting C:/Users/06411/tmp/data/t10k-labels-idx1-ubyte.gz


------------------------------------------------------------
## 2. Multi Layer

### 2-1. Hyper-parameter

In [3]:
training_epochs = 25
batch_size      = 100
learning_rate   = 0.001
display_step    = 1

### 2-2. Input

In [4]:
x = tf.placeholder("float", [None, 784])
y = tf.placeholder("float", [None, 10])

### 2-3. Variables 

### Weight

In [5]:
## For ReLU Layer

def xavier_init(n_inputs, n_outputs, uniform = True):
    if uniform:
        init_range = tf.sqrt(6.0 / (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)
    else:
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev = stddev)

In [6]:
# W1 = tf.Variable(tf.random_normal([784, 256]))
# W2 = tf.Variable(tf.random_normal([256, 256]))
# W3 = tf.Variable(tf.random_normal([256,  10]))

W1 = tf.get_variable("W1", shape = [784, 256], initializer = xavier_init(784, 256))
W2 = tf.get_variable("W2", shape = [256, 256], initializer = xavier_init(256, 256))
W3 = tf.get_variable("W3", shape = [256,  85], initializer = xavier_init(256,  85))
W4 = tf.get_variable("W4", shape = [85,   28], initializer = xavier_init(85,   28))
W5 = tf.get_variable("W5", shape = [28,   10], initializer = xavier_init(28,   10))

### bias

In [7]:
b1 = tf.Variable(tf.random_normal([256]))
b2 = tf.Variable(tf.random_normal([256]))
b3 = tf.Variable(tf.random_normal([85]))
b4 = tf.Variable(tf.random_normal([28]))
b5 = tf.Variable(tf.random_normal([10]))

### 2-4. Modeling ( + dropout)

In [8]:
dropout_rate = tf.placeholder("float")

_L1 = tf.nn.relu(tf.add(tf.matmul(x,  W1), b1))
L1  = tf.nn.dropout(_L1, dropout_rate)

_L2 = tf.nn.relu(tf.add(tf.matmul(L1, W2), b2))
L2  = tf.nn.dropout(_L2, dropout_rate)

_L3 = tf.nn.relu(tf.add(tf.matmul(L2, W3), b3))
L3  = tf.nn.dropout(_L3, dropout_rate)

_L4 = tf.nn.relu(tf.add(tf.matmul(L3, W4), b4))
L4  = tf.nn.dropout(_L4, dropout_rate)

hypothesis = tf.add(tf.matmul(L4, W5), b5)

### 2-5. Optimization (Adam)

In [9]:
cost      = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = hypothesis, labels = y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)

### 2-6. tf.Session

In [10]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(training_epochs):
        ## mini batch
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples / batch_size)

        ## mini batch training
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(optimizer,
                     feed_dict = {x:batch_xs, y:batch_ys, dropout_rate:0.7}) ## 30% drop
            avg_cost += sess.run(cost, feed_dict = {x:batch_xs, y:batch_ys, dropout_rate:0.7}) / total_batch

        ## log print
        if epoch % display_step == 0:
            print("Epoch: ", '%04d' % (epoch + 1), "cost: ", "{:.9f}".format(avg_cost))

    print("Optimization DONE!")
    print("==========================================================")

    
    ## test
    correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy: ", accuracy.eval({x:mnist.test.images,
                                       y:mnist.test.labels,
                                       dropout_rate:1}))  ## drop 없음
    print("==========================================================")

Epoch:  0001 cost:  0.848266140
Epoch:  0002 cost:  0.319201443
Epoch:  0003 cost:  0.233773562
Epoch:  0004 cost:  0.193970909
Epoch:  0005 cost:  0.167831063
Epoch:  0006 cost:  0.144860394
Epoch:  0007 cost:  0.135572017
Epoch:  0008 cost:  0.120552780
Epoch:  0009 cost:  0.114855470
Epoch:  0010 cost:  0.109902524
Epoch:  0011 cost:  0.103506947
Epoch:  0012 cost:  0.092330760
Epoch:  0013 cost:  0.088136330
Epoch:  0014 cost:  0.082558322
Epoch:  0015 cost:  0.079261278
Epoch:  0016 cost:  0.079158922
Epoch:  0017 cost:  0.080147206
Epoch:  0018 cost:  0.074454573
Epoch:  0019 cost:  0.068936523
Epoch:  0020 cost:  0.067167557
Epoch:  0021 cost:  0.068917945
Epoch:  0022 cost:  0.062487079
Epoch:  0023 cost:  0.062980401
Epoch:  0024 cost:  0.055665054
Epoch:  0025 cost:  0.056085816
Optimization DONE!
Accuracy:  0.9802


--------------------------------------------------------------------------------

*End of Code*